In [7]:
import numpy as np
import mdptoolbox
import pdb
#*****************************************************************************************************************************
# states 0-100, state '0' being the original new product. the rest of the states are divided into 5 stages (1-5) with
# a multiple of 4 states each denoted by the script following the stage. Eg 1-o is when the product is reshipped successfully, 
#1-x is product is returned by customer, 1-r the product is repaired and sent to be reshipped as a refurbished product and 
#1-e is the product is sent to be destroyed (trashed). therefore in this scheme states 1 to 4 correspond to 1-o, 1-x, 1-r and 1-e
# respectively Similarly 5-8, 9-12....17-20 correspond to stages 2,3,4 and 5 respectively ....and so on....

#******************************************************************************************************************************
S = list(range(101))
A = ["SHIP","TRASH"]
# Parameters, prob of ship back- p_s1, prob of finding a defect p_e1, initial value of product V_0, 
# costs for test ship repair and salvage, Gross Margin (GM assumed to be 70% of V_0)  and refurbished values at each stage
p_s1 = 0.02
p_e1 = 0.25
V_0 = 100
GM = 0.7*V_0
remade = 0.3*V_0
test = 0.03*V_0
ship =0.04*V_0
repair = 0.1*V_0
salvage =0.1*V_0 # positive salvage is a cost to Cisco, -ve salvage is if it gets a residual value  out of the components
vlosscoeff = 0.1 #( this is how much the value erodes per stage if the initial value was the GM, the next stage value
                # will be vlosscoeff times GM and so on. This represents the penalty caused by customer dissatisfaction expressed in 
                # monetary value. we could use any loss function which models this accurately. linear values decreases from
                # Gm to shipping cost or zero and so on)
gamma = 0.6 #( Using 1 ensures a constant refurbished value)
#******************************************************************************************************************************
Value=[]
Value.append(GM)
vthreshold = -200000 #ship 
for i in range(1, len(S)//4):
    #Value.append(max(vlosscoeff*Value[i-1],vthreshold))
    Value.append(Value[0]-10*i) # linear  (constt slope)
    #Value.append(Value[0]- 0.8*i^2) # quadratic decline
    #Value.append(Value[i-1]-0.8*i) #  linear piece wise decline (slope increases)
print(Value)
#******************************************************************************************************************************
sigma =-200000 #repair 
refurbished = []
refurbished.append(remade) # initial value of the fraction of V_0
for i in range(1,len(S)//4-1):
    #refurbished.append(max(refurbished[i-1]*gamma ,sigma)) #fractional decline
    refurbished.append(refurbished[0]-10*i) # linear decline (constt slope)
    #refurbished.append(refurbished[0]-0.8*i^2) # quadratic decline
    #refurbished.append(refurbished[i-1]-0.8*i) #  linear piece wise decline (slope increases)
print(refurbished)
#*****************************************************************************************************************************
# Prob_shipback is the return probability Assume it increases linearly (by a constant increase in probability) per stage with
# an increment of delta for eg if the prob is 2% in 1st stage (call it p_1)  then (p_5) could be 10%  in stage 5 (2%).(-1 <= delta <= 1)
Prob_shipback = np.zeros(len(S)//4)
delta= 0.0 #(Using zero ensures a constant re-shipping probability = p_s1 at every stage)
for x, val in enumerate(Prob_shipback):
    if x ==0:
        Prob_shipback[x] = p_s1
    else:
        Prob_shipback[x] = max(0,min((x*delta) + Prob_shipback[0] ,1) ) 
#print(Prob_shipback)
#*****************************************************************************************************************************
# Prob_errorfound is the chance of finding an error at a stage, give that the customer has returned a product. We can again assume it is 
#linearly decreasing per stage as above. Let rho (0< rho <= 1) be a factor at each stage of return. (rho ==1 is constant, <1 is
# decreasing you can use rho >1 if increasing as well).
rho = 1 #(Using 1 ensures a constant probability = p_e1 of finding an error at every stage)
Prob_errorfound = np.zeros(len(S)//4 -1)
Prob_errorfound[0] = p_e1
for x, val in enumerate(Prob_errorfound):
    if x !=0:
        Prob_errorfound[x] = min(rho*Prob_errorfound[x-1],1)
#print(Prob_errorfound)
#*****************************************************************************************************************************
TPM = np.zeros((len(A),len(S),len(S)))
TPM.shape
for x, val in enumerate(A):
    for y in S:
        for z in S:
            if y==0:
                if x ==0:
                    TPM[x][y][2]=Prob_shipback[y]
                    TPM[x][y][1]=1- Prob_shipback[y]
                else:
                    TPM[x][y][4]=1
            elif y%4 ==2 and y/4 <len(S)//4 -1:
                if x == 0:
                    TPM[x][y][y+1]=Prob_errorfound[int(y/4)]
                    TPM[x][y][y+3]=(1- Prob_shipback[int(y/4+1)])*(1-Prob_errorfound[int(y/4)])
                    TPM[x][y][y+4]=Prob_shipback[int(y/4+1)]*(1-Prob_errorfound[int(y/4)])
                else:
                    TPM[x][y][y+2]=1
            elif float(y/4)>float(len(S)//4 -1):
                TPM[x][y][y]=1
            elif y==z:
                TPM[x][y][y]=1
#print(TPM)
#*****************************************************************************************************************************
# Let V_0 be the initial value of the product (in $). When it is first shipped and accepted the reward is gross margin x V_0
# Subsequently the rewards depend on the state as the cost of testing repairs, salvage (if applicable) and reshipping are added, which
# diminish the value of the product as number of stages increases. The refurbished value is again a fraction of the present value of 
# the product in the previous stage
Reward = -np.zeros((len(A),len(S),len(S)))
Reward.shape
for x, val in enumerate(A):
    for y in S:
        for z in S:
            if y ==0:
                if x ==0:
                    Reward[x][y][2]=-test
                    Reward[x][y][1]= Value[0]
                else:
                    Reward[x][y][4]= -salvage
            elif y%4 ==2 and y/4 <len(S)//4 -1:
                if x == 0:
                    Reward[x][y][y+1]=-repair+ refurbished[int(y/4)]
                    Reward[x][y][y+4]=-(ship+ test)
                    Reward[x][y][y+3]=-ship+ Value[int(y/4+1)]
                else:
                    Reward[x][y][y+2]=-salvage
            


#print(Reward)
beta = 0.999999999
#******************************************************************************************************************************
#Policy Iteration
#grid = mdptoolbox.mdp.PolicyIteration(TPM, Reward, beta)
#grid.run()
#print(grid.policy) 
#print(grid.V)
#print(grid.iter) 
#print(grid.time)
#******************************************************************************************************************************
#Value iteration
grid = mdptoolbox.mdp.ValueIteration(TPM, Reward, beta)
grid.run()
print(grid.V)
policy=[]
policy.append(grid.policy[0])
for i in range(0,len(grid.policy)-1,1):
    if i%4 ==2:
         policy.append(grid.policy[i])
#strikes = (len(policy)-1-np.count_nonzero(policy))  
#print(strikes)
strikes = np.argmax(policy)
if strikes ==0:
    strikes =-1
print(policy)
print(strikes)
#******************************************************************************************************************************


[70.0, 60.0, 50.0, 40.0, 30.0, 20.0, 10.0, 0.0, -10.0, -20.0, -30.0, -40.0, -50.0, -60.0, -70.0, -80.0, -90.0, -100.0, -110.0, -120.0, -130.0, -140.0, -150.0, -160.0, -170.0]
[30.0, 20.0, 10.0, 0.0, -10.0, -20.0, -30.0, -40.0, -50.0, -60.0, -70.0, -80.0, -90.0, -100.0, -110.0, -120.0, -130.0, -140.0, -150.0, -160.0, -170.0, -180.0, -190.0, -200.0]
(69.47208121733169, 0.0, 46.604060913189166, 0.0, 0.0, 0.0, 36.60406091588156, 0.0, 0.0, 0.0, 26.604061085374465, 0.0, 0.0, 0.0, 16.604072374901712, 0.0, 0.0, 0.0, 6.604825000052425, 0.0, 0.0, 0.0, -3.34499999985, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
[0, 0, 0, 